In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.execution.arrow.enabled":"true"
         }
}

[I 2023-01-28 17:59:52,680.680 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python3", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv", "spark.sql.execution.arrow.enabled": "true"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-01-28 17:59:52,680.680 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


The kernel is successfully refreshed.

In [3]:
import pyEX as p
import requests
import json
import pandas as pd
import numpy as np
#import vectorbt as vbt
import matplotlib.pyplot as plt 
import itertools 
import datetime 
import awswrangler as wr
import vectorbt as vbt
import pyspark.sql.functions as py_f
import pyspark.sql.types as py_t
isSandbox = False
if isSandbox:
    token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    #token ='pk_2e94555e43da4135a6032433c6b18fa5'
    token ='pk_79d147436c1349f3abbec37591323e52'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe='max'

In [4]:
if False:
    fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
    bitcoin_etf_symbols = ['BITO']
    commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
    fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
    equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
    'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                          'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                          'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
    sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
    bench_symbol=['SPY','QQQ','IVV']
    all_symbols =fx_etf_symbols+bitcoin_etf_symbols+commod_etf_symbols+fi_etf_symbols+equity_etf_symbols+bench_symbol+sectors
    #all_symbols
    columns_to_capture = ['close','open','symbol']
    data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
    parquet_file_out =f"{data_folder}/etf_market_data.parquet"

In [37]:
syms=pd.DataFrame([{'symbol':'SPY'}])
get_new_data_df(syms)


https://cloud.iexapis.com/stable/stock/SPY/chart/1mm?token=pk_79d147436c1349f3abbec37591323e52&chartIEXOnly=true


,date,minute,close,open,high,low,symbol
0,2022-12-29,09:30,381.755,379.670,381.770,379.120,SPY
1,2022-12-29,10:00,382.230,381.720,382.885,381.395,SPY
2,2022-12-29,10:30,383.200,382.630,383.220,382.420,SPY
3,2022-12-29,11:00,382.840,383.100,383.200,382.275,SPY
4,2022-12-29,11:30,383.925,382.860,384.230,382.860,SPY
...,...,...,...,...,...,...,...
253,2023-01-27,13:30,406.420,406.100,406.535,406.025,SPY
254,2023-01-27,14:00,407.060,406.375,407.165,406.290,SPY
255,2023-01-27,14:30,407.440,407.145,407.580,406.930,SPY
256,2023-01-27,15:00,407.530,407.535,408.145,407.070,SPY


In [ ]:
#@py_f.pandas_udf("date timestamp, close double, open double, symbol string", py_f.PandasUDFType.GROUPED_MAP)
#@py_f.pandas_udf("symbol string", py_f.PandasUDFType.GROUPED_MAP)
def get_new_data_df(symbol):
    local_sym=str(symbol.values[0][0])
    try:
        hwm_l=highwatermark_pd_bc.value
        columns_to_capture=columns_to_capture_bc.value
        iex_base_url=iex_base_url_bc.value
        sym_pd = hwm_l.query(f"symbol=='{local_sym}'")
        if len(sym_pd)>0:
            max_date = sym_pd['max_date'].iloc[0]
        else:
            max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
        last_days=(datetime.datetime.now()-max_date).days

        pyEX_cl = p.Client(api_token=iex_token_bc.value)
        final_url = iex_base_url.format(local_sym,iex_token_bc.value,int(last_days))
        #'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        print(final_url)
        res = requests.get(final_url)
        ret_pd = pd.DataFrame(json.loads(res.text))#[columns_to_capture]
        ret_pd=ret_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")
        ret_pd['date']=pd.to_datetime(ret_pd['date'])
        ret_pd['symbol']=local_sym
        #return(ret_pd[columns_to_capture].query(f"date>'{max_date}'"))
    except:
        ret_pd=pd.DataFrame([{'date': datetime.datetime.now()-datetime.timedelta(days=365*50),
                              'minute':None,
                              'close': None, 
                              'open': None, 
                              'high': None, 
                              'low': None, 
                              'symbol': local_sym}])
    #return(ret_pd[columns_to_capture],max_date,hwm_l,local_sym,sym_pd)
    return(ret_pd[columns_to_capture])
    #return(pd.DataFrame([res.text],columns=['symbol']))
    
class MarketDataGateway():
    def __init__(self,isSandbox=False):
        self.fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
        self.bitcoin_etf_symbols = ['BITO','BTF','XBTF','BITS','GBTC','BITW','BLOK','BLCN','LEGR','SPBC','BITQ','BKCH','DAPP','BTCFX']
        self.bitcoin_equity_proxy_symbols = ['COIN','MSTR']
        self.crypto_mining_symbols = ['RIOT','CAN','HUT','HIVE','MARA','BTCM','BTBT','BITF']
        self.commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
        self.fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
        self.equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
                'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
                    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
        self.sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
        self.bench_symbol=['SPY','QQQ','IVV']
        self.all_symbols =self.fx_etf_symbols+\
                            self.bitcoin_etf_symbols+\
                            self.commod_etf_symbols+\
                            self.bitcoin_equity_proxy_symbols+\
                            self.crypto_mining_symbols+\
                            self.fi_etf_symbols+\
                            self.equity_etf_symbols+\
                            self.bench_symbol+\
                            self.sectors
        self.all_symbols_df = spark.createDataFrame(pd.DataFrame(self.all_symbols,columns=['symbol']))
        #all_symbols
        self.columns_to_capture = ['date','minute','close','open','high','low','symbol']
        self.data_folder = 's3://fsidatalake/intraday_market_data/'
        self.parquet_file_out =f"{self.data_folder}/OHLC_30m.parquet"
        if isSandbox:
            self.iex_token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
            self.iex_ver = 'sandbox'
        else:
            #self.iex_token ='pk_2e94555e43da4135a6032433c6b18fa5' pk_79d147436c1349f3abbec37591323e52
            self.iex_token ='pk_79d147436c1349f3abbec37591323e52' 
            self.iex_ver = 'stable'
        self.iex_base_url = 'https://cloud.iexapis.com/'+self.iex_ver+'/stock/{}/chart/1mm?token={}&chartIEXOnly=true'
        
    def set_highwatermarks(self):
        try:
            self.old_df = spark.read.parquet(self.parquet_file_out)
        except:
            #self.old_df = spark.createDataFrame(pd.DataFrame([{'symbol':'----','date':datetime.datetime.now()-datetime.timedelta(days=365*50)}]))
            self.old_pd=self.all_symbols_df.toPandas()
            self.old_pd['date']=datetime.datetime.now()-datetime.timedelta(days=365*50)
            self.old_df= spark.createDataFrame(self.old_pd)
        self.highwatermark_pd = self.old_df.groupby('symbol').agg(py_f.max("date").alias("max_date"),
                             py_f.min("date").alias("min_date"),
                             (py_f.max("date")-py_f.min("date")).alias("date_length")
                            ).toPandas()
    def set_new_symbol_df(self):
        self.new_sym_df=self.all_symbols_df.groupby("symbol")\
                        .applyInPandas(get_new_data_df, schema="date timestamp,minute string, close double, open double, high double,low double ,symbol string")\
                        .where('close is not null')
    def write_new_symbol_df(self):
        self.write_stats=self.new_sym_df.groupby('symbol').agg(py_f.count('symbol').alias('data_count'))
        self.new_sym_df.write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("append")\
            .parquet(self.parquet_file_out)
    def rewrite_if_dups(self):
        self.old_df_after_write = spark.read.parquet(self.parquet_file_out)
        self.dup_df = self.old_df_after_write.distinct().groupby('date','symbol').count()
        dup_count=self.dup_df.where('count>1').count()
        if dup_count>0:        
            print(f"found:{dup_count} dups; sample:{self.dup_df.show()}")
            self.old_df_after_write.groupby('date','symbol').agg(py_f.mean('close').alias('close'),py_f.mean('open').alias('open'))\
            .write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(self.parquet_file_out.replace('.parquet','_temp.parquet'))
        else:
            print(f"no dups")
        

mdg=MarketDataGateway()
mdg.set_highwatermarks()
# set bc vars
highwatermark_pd_bc =sc.broadcast(mdg.highwatermark_pd)
columns_to_capture_bc =sc.broadcast(mdg.columns_to_capture)
iex_token_bc =sc.broadcast(mdg.iex_token)
iex_base_url_bc =sc.broadcast(mdg.iex_base_url)
# set bc vars
mdg.set_new_symbol_df()
mdg.write_new_symbol_df()
print(mdg.write_stats.toPandas().sort_values(['symbol']))
#mdg.rewrite_if_dups()

In [46]:
mdg.old_df_after_write.distinct().groupby('date','minute','symbol').count().where('count>1').count()#.orderBy(['count','symbol'],ascending=False).show()

AttributeError: 'MarketDataGateway' object has no attribute 'old_df_after_write'

In [47]:
mdg.new_sym_df.groupby('symbol').agg(py_f.count('symbol').alias('data_count')).show()

+------+----------+
|symbol|data_count|
+------+----------+
|   XLI|       245|
|  PAVE|       134|
|   EFV|       233|
|   IWN|       169|
|  ARKG|       204|
|   SIL|       157|
|  GUNR|       199|
|  MLPX|        32|
|   EIS|        16|
|  EMLC|       209|
|   SPY|       256|
|   XSD|        43|
|  ICLN|       247|
|   LIT|       218|
|  BITQ|        29|
|  BITS|         3|
|   XLK|       252|
|   CGW|        69|
|  PICK|       190|
|  BLCN|        45|
+------+----------+
only showing top 20 rows



In [ ]:
#lpd=mdg.highwatermark_pd
#sym='abc'
#sym_pd = lpd.query(f"symbol=='{sym}'")['max_date']
#if len(sym_pd)>0:
#    max_date = lpd.query(f"symbol=='{sym}'")['max_date'].iloc[0]
#else:
#    max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
#last_days=(datetime.datetime.now()-max_date).days
#res_pd = pyEX_cl.chartDF(symbol=sym, timeframe=timeframe,last=last_days).query(f"date>'{(max_date)}'")
res_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")

In [12]:
#pd.DataFrame([{'symbol':'----','date':datetime.datetime.now()-datetime.timedelta(days=365*50)}])

mdg.all_symbols_df.toPandas()

,symbol
0,FXA
1,FXY
2,FXE
3,FXB
4,UUP
...,...
144,XLI
145,XLB
146,XLRE
147,XLK


In [ ]:

#base_url = 'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
#headers = {
#      'content-type': "application/json"
#  }

res = requests.get(base_url)
pd.DataFrame(json.loads(res.text))


In [ ]:
(max_date-10).strftime('%Y-%m-%d')

In [6]:
lpd

NameError: name 'lpd' is not defined